In [1]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

True

In [2]:
import os

# Set environment variables
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_PROJECT'] = 'cortex'

# Get keys from the environment
langchain_api_key = os.getenv("LANGCHAIN_API_KEY")
groq_api_key = os.getenv("GROQ_API_KEY")

if langchain_api_key:
    os.environ['LANGCHAIN_API_KEY'] = langchain_api_key
else:
    raise ValueError("LANGCHAIN_API_KEY is not set in the environment.")

if groq_api_key:
    os.environ['GROQ_API_KEY'] = groq_api_key
else:
    raise ValueError("GROQ_API_KEY is not set in the environment.")

PART 5 - MULTI QUERY

In [3]:
import os
import bs4
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_groq import ChatGroq
from langchain_huggingface import HuggingFaceEmbeddings

# Set USER_AGENT before making requests
os.environ["USER_AGENT"] = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36"
os.environ["TOKENIZERS_PARALLELISM"] = "false"

# Load Documents
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
blog_docs = loader.load()

# Split - Chunking
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=300, 
    chunk_overlap=50
)

# Make splits
splits = text_splitter.split_documents(blog_docs)

# Embed
model_name = "BAAI/bge-small-en"
model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings": True}

# Initialize HuggingFaceEmbeddings
hf_embeddings = HuggingFaceEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
)

# Index with FAISS
vectorstore = FAISS.from_documents(documents=splits, embedding=hf_embeddings)

# Retrieve
retriever = vectorstore.as_retriever()

USER_AGENT environment variable not set, consider setting it to identify your requests.


Multi-Query Prompt

In [8]:
from langchain.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
from langchain_core.output_parsers import StrOutputParser

# Disable LangSmith tracing to prevent API errors
os.environ["LANGCHAIN_TRACING_V2"] = "false" 

# Define the template for generating multiple perspectives on the user's question
template = """
You are an AI language model assistant. Your task is to generate five 
different versions of the given user question to retrieve relevant documents from a vector 
database. By generating multiple perspectives on the user question, your goal is to help
the user overcome some of the limitations of the distance-based similarity search. 

Provide these alternative questions separated by newlines.

Original question: {question}
"""

# Create the prompt template using ChatPromptTemplate
prompt_perspectives = ChatPromptTemplate.from_template(template)

from langchain_groq import ChatGroq

llm = ChatGroq(
    temperature=0,
    model="llama3-70b-8192" 
)

# Define the pipeline for generating alternative questions
generate_queries = (
    prompt_perspectives     # Use the prompt template
    | llm                   # Use the ChatGroq model with zero temperature (deterministic responses)
    | StrOutputParser()     # Parse the output into a clean format (a single string with questions separated by newlines)
    | (lambda x: "\n".join([q.strip() for q in x.split("\n") if q.strip()]))  # Clean and join questions with newlines
)

# Example: Use the pipeline to generate alternative questions for a given user question
user_question = "What are the benefits of using LangChain for NLP tasks?"
alternative_questions = generate_queries.invoke({"question": user_question})

# Print the generated alternative questions
print(alternative_questions)

Here are five alternative versions of the original question:
What are the advantages of LangChain in natural language processing applications?
How does LangChain improve the performance of NLP models?
What are the key benefits of integrating LangChain into NLP pipelines?
Can LangChain enhance the efficiency of NLP workflows can benefit from LangChain?
What are the use cases where LangChain excels in NLP tasks?
These alternative questions can help retrieve relevant documents from a vector database by providing different perspectives on the original question, which can overcome some of the limitations of similarity search.


In [9]:
from langchain.load import dumps, loads

def get_unique_union(documents: list[list]):
    """ Unique union of retrieved docs """
    # Flatten list of lists, and convert each Document to string
    flattened_docs = [dumps(doc) for sublist in documents for doc in sublist]
    # Get unique documents
    unique_docs = list(set(flattened_docs))
    # Return
    return [loads(doc) for doc in unique_docs]

# Retrieve
question = "What is task decomposition for LLM agents?"
retrieval_chain = generate_queries | retriever.map() | get_unique_union
docs = retrieval_chain.invoke({"question":question})
len(docs)

/var/folders/bg/7gm4yt417px42myyczdlqf1h0000gn/T/ipykernel_55892/2784938305.py:10: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  return [loads(doc) for doc in unique_docs]


15

In [11]:
from langchain.load import dumps, loads
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
from langchain_core.output_parsers import StrOutputParser
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough

# RAG template
template = """Answer the following question based on this context:

{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

llm = ChatGroq(model="llama3-70b-8192", temperature=0)

# Define the final RAG chain
final_rag_chain = (
    {"context": retrieval_chain, "question": itemgetter("question")}  # Ensure the mapping is correct
    | prompt
    | llm
    | StrOutputParser()
    | (lambda x: "\n".join([q.strip() for q in x.split("\n") if q.strip()]))
)

# Define the question
question = "What is task decomposition for LLM agents?"

# Invoke the chain with the question and get the result
result = final_rag_chain.invoke({"question": question})

# Print the result
print(result)

Based on the provided context, task decomposition for LLM agents is not explicitly defined. However, we can infer some information about task decomposition from the context.
In the context, we see that the AI assistant can parse user input into several tasks, and each task has a specific format with fields like "task", "id", "dep", and "args". The "dep" field denotes the id of the previous task that generates a new resource that the current task relies on.
We also see that the task MUST be selected from a list of available tasks, and there is a logical relationship between tasks, with a specific order.
Additionally, we see that the LLM distributes tasks to expert models, where the request is framed as a multiple-choice question. LLM is presented with a list of models to choose from, and task type-based filtration is needed due to the limited context length.
Therefore, we can infer that task decomposition for LLM agents involves breaking down complex tasks into smaller, more manageable 

PART 6 - RAG-FUSION

In [12]:
from langchain.prompts import ChatPromptTemplate

# RAG-Fusion: Related
template = """
You are a helpful assistant tasked with generating multiple search queries based on a single input query. 
The goal is to create queries that are semantically related to the input while capturing different aspects of the topic.

Input Query: {question}

Output: Provide exactly 4 related search queries, each on a new line.
"""
prompt_rag_fusion = ChatPromptTemplate.from_template(template)

In [26]:
from langchain_core.output_parsers import StrOutputParser
from langchain_groq import ChatGroq

# Define the query generation pipeline
generate_queries = (
    prompt_rag_fusion
    | ChatGroq(model="llama3-70b-8192", temperature=0)
    | StrOutputParser()
    | (lambda x: [
        query.strip().lstrip("1234567890. ")  # Remove numbering and extra spaces
        for query in x.split("\n") if query.strip()  # Remove empty lines
    ])
)


In [27]:
from langchain.load import dumps, loads

def reciprocal_rank_fusion(results: list[list], k=60):
    """Applies Reciprocal Rank Fusion (RRF) to combine ranked document lists."""
    fused_scores = {}

    for docs in results:
        for rank, doc in enumerate(docs):
            doc_str = dumps(doc)  # Serialize document for use as dictionary key
            fused_scores[doc_str] = fused_scores.get(doc_str, 0) + 1 / (rank + k)

    # Sort by score in descending order and deserialize documents
    return [
        (loads(doc), score) for doc, score in sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)
    ]

# RAG Fusion Retrieval Chain
retrieval_chain_rag_fusion = generate_queries | retriever.map() | reciprocal_rank_fusion

# Invoke the chain with a question
docs = retrieval_chain_rag_fusion.invoke({"question": question})

# Output the length of the fused document list
print(f"Number of retrieved documents: {len(docs)}")



Number of retrieved documents: 9


In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from operator import itemgetter

# Define your context and question prompt
template = """Answer the following question based on this context:

{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

# Assuming retrieval_chain_rag_fusion is defined earlier
# You need to pass the context from your retrieval chain and the question as the input

# Define the final RAG chain
final_rag_chain = (
    {"context": retrieval_chain_rag_fusion, 
     "question": itemgetter("question")} 
    | prompt
    | ChatGroq(model="llama3-70b-8192", temperature=0)  
    | StrOutputParser()
    | (lambda x: "\n".join([line.strip() for line in x.split("\n") if line.strip()]))
)

# Make sure the input is correctly structured
question = "What is the impact of AI on healthcare?"

# Now invoke the chain with the correctly formatted input
response = final_rag_chain.invoke({"question": question})

# Print the response
print(response)


The provided context does not directly discuss the impact of AI on healthcare. However, it does mention ChemCrow, which is a tool-augmented LLM that has been used to develop a novel anticancer drug. This suggests that AI can be used to aid in scientific discovery and potentially improve healthcare outcomes. Additionally, the context mentions health data management as one of the APIs used in the API-Bank benchmark for evaluating tool-augmented LLMs. This implies that AI can be used to manage and analyze health data, which can lead to better healthcare decisions and outcomes.


PART 7 - DECOMPOSITION

In [39]:
from langchain.prompts import ChatPromptTemplate

# Decomposition Template
template = """
You are a helpful assistant tasked with breaking down a complex input question into smaller, focused sub-questions. 
Each sub-question should address a specific aspect of the main question, allowing them to be answered in isolation.

Input: {question}

Your goal:
- Decompose the input question into exactly 3 relevant and concise sub-questions.
- Ensure each sub-question is clear, non-overlapping, and focused on a distinct aspect of the input.

Output:
1. [First sub-question]
2. [Second sub-question]
3. [Third sub-question]
"""
prompt_decomposition = ChatPromptTemplate.from_template(template)


In [41]:
from langchain_groq import ChatGroq
from langchain_core.output_parsers import StrOutputParser

# LLM
llm = ChatGroq(model="llama3-70b-8192", temperature=0)

# Chain for generating sub-questions via decomposition
generate_queries_decomposition = (
    prompt_decomposition  # The prompt template
    | llm  # Language model to process the prompt
    | StrOutputParser()  # Parses the LLM output to a string
    | (lambda x: [q.strip() for q in x.split("\n") if q.strip()])  # Clean and split into a list of sub-questions
)

# Input Question
question = "What are the main components of an LLM-powered autonomous agent system?"

# Run the Chain
try:
    questions = generate_queries_decomposition.invoke({"question": question})
    # Print the generated sub-questions
    print("\n".join(questions))
except Exception as e:
    print(f"An error occurred: {e}")

Here are the 3 sub-questions that break down the complex input question:
1. What are the key architectural components of an LLM-powered autonomous agent system, such as the role of the LLM, perception, and action modules?
2. What are the primary functions and responsibilities of the LLM within the autonomous agent system, including tasks such as reasoning, planning, and decision-making?
3. How does the autonomous agent system integrate and leverage external data sources, such as sensors, APIs, and knowledge graphs, to inform its decision-making and action-taking processes?
These sub-questions address distinct aspects of the main question, allowing them to be answered independently while still providing a comprehensive understanding of the main components of an LLM-powered autonomous agent system.


In [42]:
template = """
You are a helpful assistant tasked with answering a given question using the provided background question-answer pairs and additional context.

Question:
{question}

Background Question-Answer Pairs:
{q_a_pairs}

Additional Context:
{context}

Using the above information, provide a clear and concise answer to the question: {question}
"""

decomposition_prompt = ChatPromptTemplate.from_template(template)

In [44]:
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser

def format_qa_pair(question, answer):
    """Format a question and answer pair as a string."""
    return f"Question: {question}\nAnswer: {answer}\n"

# Initialize the language model (LLM)
llm = ChatGroq(model="llama3-70b-8192", temperature=0)

# Initialize Q&A pairs as an empty string
q_a_pairs = ""

# Loop through the list of questions and process each one
for q in questions:
    # Define the RAG chain for retrieving and answering
    rag_chain = (
        {
            "context": itemgetter("question") | retriever,  # Retrieve context for the current question
            "question": itemgetter("question"),             # Extract the current question
            "q_a_pairs": itemgetter("q_a_pairs")            # Include existing Q&A pairs
        }
        | decomposition_prompt  # Apply the decomposition prompt
        | llm                   # Use the LLM to generate an answer
        | StrOutputParser()     # Parse the output into a clean string
    )

    # Generate an answer for the current question
    answer = rag_chain.invoke({"question": q, "q_a_pairs": q_a_pairs})

    # Format the question and answer pair
    q_a_pair = format_qa_pair(q, answer)

    # Append the new Q&A pair to the existing pairs
    q_a_pairs += f"\n---\n{q_a_pair.strip()}"

# Print the final Q&A pairs (optional)
print(q_a_pairs.strip())

---
Question: Here are the 3 sub-questions that break down the complex input question:
Answer: Based on the provided background question-answer pairs and additional context, the 3 sub-questions that break down the complex input question are:

1. Specifics of the Super Mario game (e.g. level design, characters, gameplay mechanics)
2. Details about the MVC components (e.g. which components are in each file)
3. Keyboard control implementation (e.g. which keys to use, how to handle input)

These sub-questions are derived from the conversation samples provided, where the assistant clarifies the areas that need clarification and then asks specific questions to the user.
---
Question: 1. What are the key architectural components of an LLM-powered autonomous agent system, such as the role of the LLM, perception, and action modules?
Answer: Based on the provided background question-answer pairs and additional context, the key architectural components of an LLM-powered autonomous agent system ar

Answer Individually

In [45]:
from langchain import hub
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser
from langchain_groq import ChatGroq

# Pull the RAG prompt template from the hub
prompt_rag = hub.pull("rlm/rag-prompt")

def retrieve_and_rag(question, prompt_rag, sub_question_generator_chain):
    """
    Perform RAG (Retrieve and Generate) for each sub-question.
    
    Args:
        question (str): The main input question.
        prompt_rag: The RAG prompt template.
        sub_question_generator_chain: A chain that generates sub-questions.
        retriever: A retriever object to fetch relevant documents.
        llm: The language model for answering questions.

    Returns:
        tuple: A list of answers and the corresponding sub-questions.
    """
    # Generate sub-questions from the input question
    sub_questions = sub_question_generator_chain.invoke({"question": question})

    # Initialize a list to store answers for each sub-question
    rag_results = []

    # Process each sub-question
    for sub_question in sub_questions:
        # Retrieve documents relevant to the sub-question
        retrieved_docs = retriever.invoke(sub_question)

        # Generate an answer using the RAG prompt, LLM, and retrieved documents
        answer = (
            prompt_rag  # Apply the RAG prompt
            | llm       # Pass it through the language model
            | StrOutputParser()  # Parse the LLM output into a clean string
        ).invoke({"context": retrieved_docs, "question": sub_question})

        # Append the generated answer to the results list
        rag_results.append(answer)

    return rag_results, sub_questions

# Wrap the retrieval and RAG process in a RunnableLambda for integration into a chain
answers, questions = retrieve_and_rag(question, prompt_rag, generate_queries_decomposition)

In [46]:
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import ChatPromptTemplate

def format_qa_pairs(questions, answers):
    """
    Format a list of questions and their corresponding answers into a structured string.

    Args:
        questions (list): List of questions.
        answers (list): List of answers corresponding to the questions.

    Returns:
        str: Formatted string of Q&A pairs.
    """
    return "\n\n".join(
        f"Question {i}: {q}\nAnswer {i}: {a}" for i, (q, a) in enumerate(zip(questions, answers), start=1)
    ).strip()

# Format the Q&A pairs to create the context
context = format_qa_pairs(questions, answers)

# Define the prompt template for synthesizing an answer
template = """
Here is a set of Q+A pairs:

{context}

Use these to synthesize a clear and concise answer to the following question:

{question}
"""
prompt = ChatPromptTemplate.from_template(template)

# Define the RAG chain
final_rag_chain = (
    prompt  # Use the prompt template
    | llm  # Pass the input through the language model
    | StrOutputParser()  # Parse the output into a clean string
)

# Invoke the chain with the given context and question
result = final_rag_chain.invoke({"context": context, "question": question})

# Output the result
print(result)


Based on the provided Q&A pairs, here is a clear and concise answer to the question:

The main components of an LLM-powered autonomous agent system include:

1. The LLM (Large Language Model) as the agent's core controller, responsible for task decomposition, planning, and decision-making.
2. Planning component, which involves subgoal decomposition, reflection, and refinement, enabling the agent to break down complex tasks into smaller, manageable subgoals.
3. Perception module, which integrates and leverages external data sources, such as sensors, APIs, and knowledge graphs, through its retrieval model, to inform the agent's behavior.
4. Action module, which takes action based on the decisions made by the LLM.
5. Reflection mechanism, which synthesizes memories into higher-level inferences over time and guides the agent's future behavior.

These components work together to enable the autonomous agent system to make informed decisions and take actions in a complex environment.


PART 8 - STEP BACK

In [47]:
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate

# Define few-shot examples
examples = [
    {
        "input": "Could the members of The Police perform lawful arrests?",
        "output": "What can the members of The Police do?",
    },
    {
        "input": "Jan Sindel was born in what country?",
        "output": "What is Jan Sindel’s personal history?",
    },
]

# Transform examples into example messages using a message prompt template
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)

# Create a few-shot prompt template with the example prompt and examples
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

# Combine the system message, few-shot examples, and user question into the final prompt
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are an expert in world knowledge. Your task is to reframe a specific question into a more generic, step-back question that is easier to answer. Here are a few examples:""",
        ),
        # Add few-shot examples
        few_shot_prompt,
        # Add the user question
        ("user", "{question}"),
    ]
)


In [50]:
generate_queries_step_back = prompt | ChatGroq(model="llama3-70b-8192", temperature=0) | StrOutputParser()
question = "What is task decomposition for LLM agents?"
generate_queries_step_back.invoke({"question": question})

'How do artificial intelligence systems break down complex tasks?'

In [52]:
# Define the response prompt template
response_prompt_template = """
You are an expert in world knowledge. I will ask you a question. Your response should be comprehensive and aligned with the provided contexts if they are relevant. Otherwise, ignore them if they are not relevant.

# Normal Context:
{normal_context}

# Step-Back Context:
{step_back_context}

# Original Question:
{question}

# Answer:
"""

# Create the response prompt
response_prompt = ChatPromptTemplate.from_template(response_prompt_template)

# Define the chain for context retrieval and response generation
chain = (
    {
        # Retrieve normal context using the original question
        "normal_context": RunnableLambda(lambda x: x["question"]) | retriever,
        # Retrieve step-back context using the step-back question
        "step_back_context": generate_queries_step_back | retriever,
        # Pass through the original question
        "question": lambda x: x["question"],
    }
    | response_prompt  # Generate a prompt with the retrieved context and question
    | ChatGroq(model="llama3-70b-8192", temperature=0)  # Generate the response using the LLM
    | StrOutputParser()  # Parse the response into a clean string
)

# Invoke the chain with the input question
result = chain.invoke({"question": question})

# Print the generated response
print(result)


Task decomposition for LLM (Large Language Model) agents refers to the process of breaking down complex tasks into smaller, manageable sub-tasks or steps. This is a crucial component of planning in LLM-powered autonomous agent systems.

Task decomposition can be achieved through various methods, including:

1. **Chain of Thought (CoT)**: A standard prompting technique that instructs the model to "think step by step" to decompose hard tasks into smaller and simpler steps. CoT transforms big tasks into multiple manageable tasks, providing insight into the model's thinking process**.
3. **Tree of Thoughts (ToT)**: An extension of CoT that explores multiple reasoning possibilities at each step, creating a tree structure. The search process can be done using Breadth-First Search (BFS) or Depth-First Search (DFS) with each state evaluated by a classifier (via a prompt) or majority vote.
4. **Simple prompting**: Using simple prompts like "Steps for XYZ.\n1.", "What are the subgoals for achiev

PART 9 - HyDE

In [53]:
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_groq import ChatGroq

# Define the HyDE document generation template
template = """
Please write a scientific paper passage to answer the following question:

Question: {question}

Passage:
"""

# Create a prompt template using ChatPromptTemplate
prompt_hyde = ChatPromptTemplate.from_template(template)

# Define the document generation chain
generate_docs_for_retrieval = (
    prompt_hyde  # Use the HyDE prompt template
    | ChatGroq(model="llama3-70b-8192", temperature=0)  # Generate the document using the language model
    | StrOutputParser()  # Parse the LLM output into a clean string
)

# Define the question
question = "What is task decomposition for LLM agents?"

# Invoke the chain to generate the passage
generated_passage = generate_docs_for_retrieval.invoke({"question": question})

# Print the generated passage
print(generated_passage)


Here is a scientific paper passage that answers the question:

**Task Decomposition for Large Language Models (LLMs) Agents**

In the realm of artificial intelligence, Large Language Models (LLMs) have emerged as powerful tools for tackling complex tasks, such as natural language processing, text generation, and dialogue systems. However, as the complexity of these tasks continues to grow, it has become increasingly important to develop strategies for decomposing them into more manageable sub-tasks. This process, known as task decomposition, is a crucial step in enabling LLM agents to reason about and execute complex tasks in a more efficient and effective manner.

In the context of LLM agents, task decomposition involves identifying the constituent sub-tasks, or "micro-tasks," that comprise a larger task. These micro-tasks are typically smaller, more focused, and more easily executable by the LLM agent. For example, in a task such as "write a short story about a character who learns t

In [54]:
# Retrieve
retrieval_chain = generate_docs_for_retrieval | retriever 
retrieved_docs = retrieval_chain.invoke({"question":question})
retrieved_docs

[Document(id='fcd3f123-3ba8-4a25-a748-1d5d82d3a970', metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='Fig. 1. Overview of a LLM-powered autonomous agent system.\nComponent One: Planning#\nA complicated task usually involves many steps. An agent needs to know what they are and plan ahead.\nTask Decomposition#\nChain of thought (CoT; Wei et al. 2022) has become a standard prompting technique for enhancing model performance on complex tasks. The model is instructed to “think step by step” to utilize more test-time computation to decompose hard tasks into smaller and simpler steps. CoT transforms big tasks into multiple manageable tasks and shed lights into an interpretation of the model’s thinking process.\nTree of Thoughts (Yao et al. 2023) extends CoT by exploring multiple reasoning possibilities at each step. It first decomposes the problem into multiple thought steps and generates multiple thoughts per step, creating a tree structure. The sear

In [55]:
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_groq import ChatGroq

# Define the RAG prompt template
template = """
Answer the following question based on the provided context:

Context:
{context}

Question:
{question}
"""

# Create a prompt from the template
prompt = ChatPromptTemplate.from_template(template)

# Define the RAG chain
final_rag_chain = (
    prompt  # Generate the prompt using the template
    | ChatGroq(model="llama3-70b-8192", temperature=0)  # Use the LLM to process the prompt
    | StrOutputParser()  # Parse the output into a clean string
)

# Invoke the RAG chain with the context & question
result = final_rag_chain.invoke({
    "context": retrieved_docs,  # The retrieved documents as context
    "question": question  # The question to be answered
})

# Print the result
print(result)

According to the provided context, task decomposition for LLM agents is the process of breaking down complex tasks into smaller, manageable subgoals. This can be done in three ways:

1. By using simple prompting, such as "Steps for XYZ.\n1.", "What are the subgoals for achieving XYZ?"
2. By using task-specific instructions, such as "Write a story outline." for writing a novel.
3. With human inputs.

Additionally, techniques like Chain of Thought (CoT) and Tree of Thoughts can be used to enhance task decomposition. CoT involves instructing the model to "think step by step" to decompose hard tasks into smaller and simpler steps. Tree of Thoughts extends CoT by exploring multiple reasoning possibilities at each step, creating a tree structure.
